In [ ]:
!pip install -q transformers[torch] datasets

In [20]:
!pip install -q bitsandbytes trl peft

In [ ]:
!pip install flash-attn --no-build-isolation

In [1]:
import io

# 使用 Windows-1252 编码读取原始文件
with io.open("training_dataset.jsonl", "r", encoding="Windows-1252") as f:
    data = f.read()

# 以 UTF-8 编码写入新文件
with io.open("training_dataset_utf8.jsonl", "w", encoding="utf-8") as f:
    f.write(data)

# 使用转换后的文件加载数据集
from datasets import load_dataset
dataset = load_dataset("json", data_files="training_dataset_utf8.jsonl")
print(dataset)


C:\Users\aaron\anaconda3\envs\Llata\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 967 examples [00:00, 161165.54 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 967
    })
})


In [14]:
example = dataset["train"][3]
print(example["input"])

user: You are a math solving assistant and need to use SymPy to help with calculations. Follow these step-by-step instructions using only symbols and expression (expr) operations:
Step 1: Output only the symbol 'x'.
Step 2: Output a mathematical expression involving 'x' (e.g., x**2 + 2*x + 1). Do not include variable assignment; only provide the expression itself.
Step 3: Output only the name of a SymPy function you intend to use on the expression (e.g., expand, solve, Eq).
Step 4: Output the arguments you would pass to that function, separated by commas (e.g., expr, x).

step 5: Wheather you need to continue the next step(e.g., expr,x).

Your final output should be a single JSON block like the example below:
"```json
        {
            "symbol": "x",
            "expr": "x+8",
            "sympy_function": "solve",
            "function_args": "expr, x"
            "require_next_step": "True"
        }

        ```
        "Remember you don't need to do any additional calculation, 

In [2]:
# 使用 chardet 库检测文件编码
import chardet

with open("training_dataset.jsonl", "rb") as f:
    result = chardet.detect(f.read())
print(result)  # 打印检测到的编码信息


ModuleNotFoundError: No module named 'chardet'

In [3]:
# 假设你已经导入并加载了 dataset 以及 messages_to_string 函数
# 将 messages 拼接为单个字符串
def messages_to_string(messages):
    """
    将 messages 转换为单个字符串，保留对话上下文。
    """
    context = ""
    for message in messages:
        # 拼接角色和内容
        context += f"{message['role']}: {message['content']}\n"
    return context.strip()  # 移除最后一个多余的换行符
def process_example(example):
    # 取出原始 input 内容
    question_prompt = example["input"]
    # 使用 messages_to_string 函数处理
    messages = messages_to_string([
        {"role": "user", "content": question_prompt}
    ])
    # 更新 input 字段
    example["input"] = messages
    return example

# 对 train 分割区的每个样本应用处理函数
dataset["train"] = dataset["train"].map(process_example)


Map: 100%|██████████| 967/967 [00:00<00:00, 26862.52 examples/s]


In [4]:
example = dataset["train"][3]
print(example["input"])

user: You are a math solving assistant and need to use SymPy to help with calculations. Follow these step-by-step instructions using only symbols and expression (expr) operations:
Step 1: Output only the symbol 'x'.
Step 2: Output a mathematical expression involving 'x' (e.g., x**2 + 2*x + 1). Do not include variable assignment; only provide the expression itself.
Step 3: Output only the name of a SymPy function you intend to use on the expression (e.g., expand, solve, Eq).
Step 4: Output the arguments you would pass to that function, separated by commas (e.g., expr, x).

step 5: Wheather you need to continue the next step(e.g., expr,x).

Your final output should be a single JSON block like the example below:
"```json
        {
            "symbol": "x",
            "expr": "x+8",
            "sympy_function": "solve",
            "function_args": "expr, x"
            "require_next_step": "True"
        }

        ```
        "Remember you don't need to do any additional calculation, 

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=False)
# model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=False)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# # Set reasonable default for models without max length
# if tokenizer.model_max_length > 100_000:
#   tokenizer.model_max_length = 2048

# # Set chat template
# DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
# tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

C:\Users\aaron\anaconda3\envs\Llata\lib\site-packages\transformers\models\auto\tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [8]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
# quantization_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype="torch.bfloat16",
# )
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    # attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    # quantization_config=quantization_config,
)

In [20]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments
def formatting_prompts_func(example):
    # 确保 example 是一个包含 "input" 和 "output" 键的字典
    input_text = example["input"]
    output_text = example["output"]
    
    # 拼接为指定格式的文本
    text = f"### Question: {input_text}\n### Answer: {output_text}"
    
    # 可选：打印输出以调试
    print(text)
    
    return text

# path where the Trainer will save its checkpoints and logs
output_dir = 'data/zephyr-7b-sft-lora'

# based on config
training_args = TrainingArguments(
    # fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    evaluation_strategy="no",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=4, # originally set to 8
    per_device_train_batch_size=4, # originally set to 8
    # push_to_hub=True,
    # hub_model_id="zephyr-7b-sft-lora",
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

trainer = SFTTrainer(
        model=model_name,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=dataset["train"],
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        formatting_func=formatting_prompts_func,
        max_seq_length=tokenizer.model_max_length,
    )

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.39it/s]


In [21]:
train_result = trainer.train()

***** Running training *****
  Num examples = 2
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 128
  Total optimization steps = 1
  Number of trainable parameters = 36,700,160


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 134486163456 bytes.

In [ ]:
metrics = train_result.metrics
max_train_samples = training_args.max_train_samples if training_args.max_train_samples is not None else len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
trainer.save_model(output_dir)